In [96]:
import pymssql
import os
from dotenv import load_dotenv


In [97]:
load_dotenv()

server = os.getenv('DB_SERVER')
port = os.getenv('DB_PORT')
database = os.getenv('DB_NAME')
username = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')

conn = pymssql.connect(
    server=server,
    user=username,
    password=password,
    database=database,
    port=port
)

cursor = conn.cursor()
print("✅ Conexión establecida")


✅ Conexión establecida


In [98]:
drop_query = "DROP TABLE IF EXISTS Forecast"
cursor.execute(drop_query)
conn.commit()
print("🗑️ Tabla anterior eliminada")


🗑️ Tabla anterior eliminada


In [99]:
create_query = """
CREATE TABLE Forecast (
    id INT PRIMARY KEY IDENTITY(1,1),
    municipio NVARCHAR(50) NOT NULL,
    datetime_inicio DATETIME NOT NULL,
    wind_speed_input NVARCHAR(MAX) NOT NULL,
    wind_speed_output NVARCHAR(MAX) NOT NULL,
    created_at DATETIME DEFAULT GETDATE(),
    UNIQUE(municipio, datetime_inicio)
);

CREATE INDEX idx_municipio_datetime ON Forecast(municipio, datetime_inicio);
CREATE INDEX idx_created_at ON Forecast(created_at);
"""

cursor.execute(create_query)
conn.commit()
print("✅ Tabla Forecast creada")


✅ Tabla Forecast creada



📋 Estructura de la tabla:
  • id: int(4)
  • municipio: nvarchar(100)
  • datetime_inicio: datetime(8)
  • wind_speed_input: nvarchar(MAX)
  • wind_speed_output: nvarchar(MAX)
  • created_at: datetime(8)



🔍 Verificación de campos:
  ✅ id
  ✅ municipio
  ✅ datetime_inicio
  ✅ wind_speed_input
  ✅ wind_speed_output
  ✅ created_at

✅ Estructura correcta



✅ Conexión cerrada


In [103]:
import pymssql
import os
from dotenv import load_dotenv


In [104]:
load_dotenv()

server = os.getenv('DB_SERVER')
port = os.getenv('DB_PORT')
database = os.getenv('DB_NAME')
username = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')


In [105]:
conn = pymssql.connect(
    server=server,
    port=port,
    user=username,
    password=password,
    database=database,
    autocommit=True
)

cursor = conn.cursor()
print("✅ Conexión establecida")

✅ Conexión establecida


In [106]:
drop_table_query = """
IF EXISTS (SELECT * FROM sys.tables WHERE name = 'Forecast')
BEGIN
    DROP TABLE Forecast;
    PRINT 'Tabla Forecast eliminada';
END
ELSE
BEGIN
    PRINT 'La tabla Forecast no existe';
END
"""

cursor.execute(drop_table_query)
conn.commit()
print("✅ Tabla verificada y eliminada si existía")


✅ Tabla verificada y eliminada si existía


In [107]:
create_table_query = """
IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'Forecast')
BEGIN
    CREATE TABLE Forecast (
        id INT PRIMARY KEY IDENTITY(1,1),
        municipio NVARCHAR(50) NOT NULL,
        datetime DATETIME NOT NULL,
        wind_speed_input NVARCHAR(MAX) NOT NULL,
        wind_speed_output FLOAT NOT NULL,
        created_at DATETIME DEFAULT GETDATE(),
        year INT NOT NULL,
        month INT NOT NULL,
        day INT NOT NULL,
        hour INT NOT NULL
    );
    
    CREATE INDEX idx_municipio_datetime ON Forecast(municipio, datetime);
    CREATE INDEX idx_municipio_date ON Forecast(municipio, year, month, day, hour);
    CREATE INDEX idx_created_at ON Forecast(created_at);
    
    PRINT 'Tabla Forecast creada exitosamente';
END
ELSE
BEGIN
    PRINT 'La tabla Forecast ya existe';
END
"""

cursor.execute(create_table_query)
conn.commit()
print("✅ Tabla Forecast creada o verificada")


✅ Tabla Forecast creada o verificada


In [108]:
verify_query = """
SELECT 
    c.name AS ColumnName,
    t.name AS DataType,
    c.max_length AS MaxLength,
    c.is_nullable AS IsNullable
FROM sys.columns c
INNER JOIN sys.types t ON c.user_type_id = t.user_type_id
WHERE c.object_id = OBJECT_ID('Forecast')
ORDER BY c.column_id
"""

cursor.execute(verify_query)
columns = cursor.fetchall()

print("\n📊 Estructura de la tabla Forecast:\n")
print(f"{'Column':<30} {'Type':<15} {'Nullable':<10}")
print("="*60)
for col in columns:
    print(f"{col[0]:<30} {col[1]:<15} {col[3]:<10}")

expected_fields = ['id', 'municipio', 'datetime', 'wind_speed_input', 
                   'wind_speed_output', 'created_at', 'year', 'month', 'day', 'hour']
actual_fields = [col[0] for col in columns]

print("\n✅ VERIFICACIÓN DE CAMPOS:")
print("-" * 60)
all_present = all(field in actual_fields for field in expected_fields)
if all_present:
    print("✓ Todos los campos requeridos están presentes")
    for field in expected_fields:
        print(f"  ✓ {field}")
else:
    print("⚠️  Faltan algunos campos:")
    for field in expected_fields:
        if field in actual_fields:
            print(f"  ✓ {field}")
        else:
            print(f"  ✗ {field} - FALTANTE")


📊 Estructura de la tabla Forecast:

Column                         Type            Nullable  
id                             int             0         
municipio                      nvarchar        0         
datetime                       datetime        0         
wind_speed_input               nvarchar        0         
wind_speed_output              float           0         
created_at                     datetime        1         
year                           int             0         
month                          int             0         
day                            int             0         
hour                           int             0         

✅ VERIFICACIÓN DE CAMPOS:
------------------------------------------------------------
✓ Todos los campos requeridos están presentes
  ✓ id
  ✓ municipio
  ✓ datetime
  ✓ wind_speed_input
  ✓ wind_speed_output
  ✓ created_at
  ✓ year
  ✓ month
  ✓ day
  ✓ hour


In [109]:
indexes_query = """
SELECT 
    i.name AS IndexName,
    STRING_AGG(c.name, ', ') WITHIN GROUP (ORDER BY ic.key_ordinal) AS Columns
FROM sys.indexes i
INNER JOIN sys.index_columns ic ON i.object_id = ic.object_id AND i.index_id = ic.index_id
INNER JOIN sys.columns c ON ic.object_id = c.object_id AND ic.column_id = c.column_id
WHERE i.object_id = OBJECT_ID('Forecast')
    AND i.name IS NOT NULL
GROUP BY i.name
"""

cursor.execute(indexes_query)
indexes = cursor.fetchall()

print("\n🔍 Índices de la tabla Forecast:\n")
for idx in indexes:
    print(f"  • {idx[0]}: {idx[1]}")



🔍 Índices de la tabla Forecast:

  • idx_created_at: created_at
  • idx_municipio_date: municipio, year, month, day, hour
  • idx_municipio_datetime: municipio, datetime
  • PK__Forecast__3213E83FD79518E3: id


In [110]:
print("\n" + "="*60)
print("📋 RESUMEN DE LA TABLA FORECAST")
print("="*60)
print("\n✓ Campos principales:")
print("  • id: ID único auto-incremental")
print("  • municipio: Nombre del municipio")
print("  • datetime: Fecha y hora completa de la predicción")
print("  • wind_speed_input: Velocidad del viento de entrada (m/s)")
print("  • wind_speed_output: Velocidad del viento predicha (m/s)")
print("  • created_at: Timestamp de creación del registro")
print("  • year: Año de la predicción")
print("  • month: Mes de la predicción (1-12)")
print("  • day: Día de la predicción (1-31)")
print("  • hour: Hora de la predicción (0-23)")
print("\n✓ Índices optimizados:")
print("  • idx_municipio_datetime: Búsqueda rápida por municipio + datetime")
print("  • idx_municipio_date: Búsqueda rápida por municipio + año/mes/día/hora")
print("  • idx_created_at: Obtener predicciones más recientes")
print("\n✓ Tabla lista para almacenar predicciones de viento")
print("="*60)



📋 RESUMEN DE LA TABLA FORECAST

✓ Campos principales:
  • id: ID único auto-incremental
  • municipio: Nombre del municipio
  • datetime: Fecha y hora completa de la predicción
  • wind_speed_input: Velocidad del viento de entrada (m/s)
  • wind_speed_output: Velocidad del viento predicha (m/s)
  • created_at: Timestamp de creación del registro
  • year: Año de la predicción
  • month: Mes de la predicción (1-12)
  • day: Día de la predicción (1-31)
  • hour: Hora de la predicción (0-23)

✓ Índices optimizados:
  • idx_municipio_datetime: Búsqueda rápida por municipio + datetime
  • idx_municipio_date: Búsqueda rápida por municipio + año/mes/día/hora
  • idx_created_at: Obtener predicciones más recientes

✓ Tabla lista para almacenar predicciones de viento


In [111]:
print("\n💡 Ejemplo de inserción:")
print("-" * 60)
print("""
INSERT INTO Forecast (
    municipio, datetime, wind_speed_input, wind_speed_output,
    year, month, day, hour
)
VALUES (
    'riohacha', '2026-01-04 14:00:00', 7.2, 7.5,
    2026, 1, 4, 14
);
""")

print("\n💡 Ejemplo de consulta:")
print("-" * 60)
print("""
SELECT 
    municipio, datetime, 
    wind_speed_input, wind_speed_output,
    created_at
FROM Forecast
WHERE municipio = 'riohacha'
    AND year = 2026
    AND month = 1
    AND day = 4
ORDER BY hour;
""")



💡 Ejemplo de inserción:
------------------------------------------------------------

INSERT INTO Forecast (
    municipio, datetime, wind_speed_input, wind_speed_output,
    year, month, day, hour
)
VALUES (
    'riohacha', '2026-01-04 14:00:00', 7.2, 7.5,
    2026, 1, 4, 14
);


💡 Ejemplo de consulta:
------------------------------------------------------------

SELECT 
    municipio, datetime, 
    wind_speed_input, wind_speed_output,
    created_at
FROM Forecast
WHERE municipio = 'riohacha'
    AND year = 2026
    AND month = 1
    AND day = 4
ORDER BY hour;



In [112]:
count_query = "SELECT COUNT(*) as total FROM Forecast"
cursor.execute(count_query)
result = cursor.fetchone()

print("\n" + "="*60)
print("✅ VERIFICACIÓN FINAL")
print("="*60)
print(f"Total de registros en la tabla: {result[0]}")
print("\n✓ La tabla está creada y lista para recibir datos")
print("="*60)



✅ VERIFICACIÓN FINAL
Total de registros en la tabla: 0

✓ La tabla está creada y lista para recibir datos


In [113]:
cursor.close()
conn.close()
print("\n✅ Conexión cerrada")



✅ Conexión cerrada
